## Computer Vision PA #2: Structure from Motion (SfM)
20195003 고강빈

In [4]:
import numpy as np
import cv2 as cv

In [9]:
img1 = cv.imread('./SfM/Data/sfm03.jpg')
img2 = cv.imread('./SfM/Data/sfm04.jpg')

# create SIFT instance
sift = cv.xfeatures2d.SIFT_create()

# detect and compute keypoints
img1_kp, img1_des = sift.detectAndCompute(img1, None)
img2_kp, img2_des = sift.detectAndCompute(img2, None)

img1_drawKps = cv.drawKeypoints(img1, img1_kp, None)
img2_drawKps = cv.drawKeypoints(img2, img2_kp, None)

# save result
cv.imwrite('sift_keypoints.jpg',img1_drawKps)

print(img1_des.shape)
print(img1_des)

(37935, 128)
[[  1.   0.   0. ...   1.   7.   7.]
 [  3.   1.   2. ...   3.   1.   1.]
 [  0.   1.   2. ...   0.   0.   0.]
 ...
 [  1.   0.   0. ...   2.   0.   1.]
 [  0.   0.   0. ...  17.  11.  16.]
 [145.  10.   2. ...   0.   0.   0.]]


In [10]:
# Brute force matching with k=2
bf = cv.BFMatcher()
matches = bf.knnMatch(img1_des, img2_des, k=2)

# Ratio test and retrieval of indices
matches_good = [m1 for m1, m2 in matches if m1.distance < 0.75*m2.distance]

sorted_matches = sorted(matches_good, key=lambda x: x.distance)
res = cv.drawMatches(img1, img1_kp, img2, img2_kp, sorted_matches, img2, flags=2) 

cv.imwrite('sift_bfMatcher.jpg',res)
print(len(sorted_matches))

1895


In [11]:
# print(res.shape)
# print(res)

In [13]:
# queryIdx : 1번 이미지의 feature point index
# trainIdx : 2번 이미지의 feature point index
query_idx = [match.queryIdx for match in matches_good]
train_idx = [match.trainIdx for match in matches_good]

#Getting float based points from good matches
p1 = np.float32([img1_kp[ind].pt for ind in query_idx])
p2 = np.float32([img2_kp[ind].pt for ind in train_idx])

print(p1.shape)
print(p1[:3])

img1_pts = np.array([img1_kp[m.queryIdx].pt for m in matches_good]).reshape(-1, 1, 2).astype(np.float32) # 픽셀 좌표
img2_pts = np.array([img2_kp[m.trainIdx].pt for m in matches_good]).reshape(-1, 1, 2).astype(np.float32)

print(img1_pts.shape)
print(img1_pts[:3])

(1895, 2)
[[  24.099754 3219.431   ]
 [ 298.76056  2282.336   ]
 [ 330.07367  2516.3718  ]]
(1895, 1, 2)
[[[  24.099754 3219.431   ]]

 [[ 298.76056  2282.336   ]]

 [[ 330.07367  2516.3718  ]]]


In [14]:
def randomsample(p1, p2):
    concat = np.concatenate((p1, p2), axis=1)
    rand_mat = concat[np.random.randint(concat.shape[0], size = len(concat)), :]
    p1s = rand_mat[:, :2]
    p2s = rand_mat[:, 2:]
    return p1s, p2s

In [15]:
def RANSAC(p1, p2, iteration):
    
    b_inlier = np.array([[]]) # best inlier
    b_E = None # best Essential Matrix
    tmp_inlier_len = 0 # tmp inlier의 개수

    for i in range(iteration):
        
        # choice random sample
        p1s, p2s = randomsample(p1, p2)
        
        # 5-point algorithm (with intrinsic parameter, epipolar constraint = 0, no RANSAC) 
        cur_E, cur_inlier = cv.findEssentialMat(p1s, p2s, focal=3092.8, pp=(2016, 1512),
                                                 maxIters = 0, threshold=0.1)

        # inlier 추출
        inlier_idx = np.where(cur_inlier==1)
        pts1 = np.vstack([p1[i] for i in inlier_idx[0]])
        pts2 = np.vstack([p2[i] for i in inlier_idx[0]])
        
        if cur_E is None:
            continue

        # inlier 수가 가장 많은 최적의 E 추출
        if len(pts1) > tmp_inlier_len: # 현재 inlier 수 > 가장 많은 inlier 수
            b_E = cur_E 
            b_inlier = cur_inlier 
            tmp_inlier_len = len(pts1)

    return b_E

In [ ]:
# MAXITER = 100
# best_inlier = np.array([[]])
# best_E = None
# inlier_len_temp = 0

# for i in range(MAXITER):
    
#     # select random points
#     p1s, p2s = randomsample(p1, p2)
    
#     # 5-points algorithm
    

In [17]:
# import matlab.engine
# eng = matlab.engine.start_matlab()
# eng.addpath("./SfM/Step2/")  # 'calibrated_fivepoint.m'가 위치한 경로
# for i in range(100):
#     a = np.random.rand(3, 5).tolist()
#     a = matlab.double(a)
#     b = np.random.rand(3, 5).tolist()
#     b = matlab.double(b)
#     E = eng.calibrated_fivepoint(a, b)
#     print(np.asarray(E))